1. 채무자키, 메모내용 읽기, 참조파일(전체원데이터, 보증인, 법조치 파산건 새로 다운) 읽기
2. 미종결 계좌가 있는 채무자키만 남기기
3. 파산사건번호 있는 메모추출(채무자키, 메모내용), 관할법원, 사건번호열 추가, 채무자이름열, 새채무자키 + 이름 + 사건번호 중복제거
4. 모든 참조 파일에 채무자키 대신 새채무자키
5. 법조치 기등록건 제외(새채무자키+사건번호--관할법원은 축어 많으니 제외)
6. 채무자키를 통해 채무자들 이름 병합 <------채무자, 보증인조회새창
7. 해당 메모에서 '차주','보증인','채무자이름'을 추가로 확인하여 대상자구분작성, 대표채무자키/보증인키 병합

### import 및 파일읽기

In [4]:
import os, re
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")
from datetime import datetime

memo_dtype = {'채무자키':str, '메모키':str, "구분일":str}
momo_cols = ["메모키","채무자키","메모내용","등록일자"]
파산_cols = ["채무자키","관할법원","사건번호","법조치구분","법조치세부","사건구분","종국결과","비고","취하상태","접수일","종료일","종국결과일","대상자구분","대상자명"]

In [18]:
# # 메모 ###################################################
# 메모_파산_ori = pd.read_excel(r"C:\Users\DATA\Desktop\파산모음\메모조회새창_20240118_1802_파산.xlsx", dtype=memo_dtype)[momo_cols]
# 메모_하단_ori = pd.read_excel(r"C:\Users\DATA\Desktop\파산모음\메모조회새창_20240118_1803_하단.xlsx", dtype=memo_dtype)[momo_cols]
# 메모_하면_ori = pd.read_excel(r"C:\Users\DATA\Desktop\파산모음\메모조회새창_20240118_1803_하단.xlsx", dtype=memo_dtype)[momo_cols]

# 합치기, 중복제거, 등록일자 전처리, to pkl
# 메모_ori = pd.concat([메모_파산_ori, 메모_하단_ori, 메모_하면_ori], axis=0)
# 메모_ori["등록일자"] = 메모_ori.등록일자.str.extract(r'(\d{4,4}-\d\d-\d\d)')
# 메모_ori = 메모_ori.drop_duplicates(subset=["메모키"], keep='first', inplace=False, ignore_index=True)
# 메모_ori.to_pickle(r"C:\Users\DATA\Desktop\파산모음\메모.pkl")

# 파산 엑셀 to pkl
# event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
# 파산법조치 = pd.read_excel(r"C:\Users\DATA\Desktop\파산모음\파산모음_법조치조회새창_20240530_1804.xlsx", dtype=event_dtype)
# 파산법조치.to_pickle(r"C:\Users\DATA\Desktop\파산모음\파산모음.pkl")

In [5]:
# 메모, 파산기등록건 법조치
메모 = pd.read_pickle(r"C:\Users\DATA\Desktop\파산모음\메모.pkl")
파산등록건 = pd.read_pickle(r"C:\Users\DATA\Desktop\파산모음\파산모음.pkl")[파산_cols]
# 참조파일 #################################################
base_dir = r"D:\3.자산\전산 dataset\솔림\240528\pkl"
raw_data = pd.read_pickle(join(base_dir,"2.솔림_전체_원데이터_240528_by새채무자키.pkl"))
grt = pd.read_pickle(join(base_dir, "보증인새창_20240528_1729.pkl"))

### 미종결건만 남기고 새채무자키 추가하기

In [12]:
# 미종결 계좌가 있는 채무자키만 남기기
미종결 = raw_data.query('새채무상태 != "종료"').copy()
# 미종결에서 채무자키 중복제거
미종결 = 미종결.drop_duplicates(subset=["채무자키"], ignore_index=True)

print("미종결메모----------")
# 메모 미종결건만 남기고 새채무자키 붙이기
미종결메모 = 메모.merge(미종결[["채무자키", "새채무자키"]], how='inner') # 기본값이 inner
print("전체메모건수", len(메모), "미종결메모건수",len(미종결메모))

# 파산등록건에 새채무자키 붙이기
print("파산등록건-----------")
print("전체", len(파산등록건))
파산등록건 = 파산등록건.merge(미종결[["채무자키","새채무자키"]], how='inner')
print("미종결건", len(파산등록건))
# 파산등록건 중 새채무자키와 사건번호 중복제거
파산등록건.drop_duplicates(subset=["새채무자키", "사건번호"], inplace=True, ignore_index=True)
print("새채무자키,사건번호 중복건 제거", len(파산등록건))

미종결메모----------
전체메모건수 20893 미종결메모건수 6572
파산등록건-----------
전체 888
미종결건 888
새채무자키,사건번호 중복건 제거 705


In [36]:
# 파산사건번호 컴파일
y_10 = str(datetime.today().year)[2] #23년이면 2
y_10_before = str(int(y_10)-1) #23년이면 2-1 = 1
y_1 = str(datetime.today().year)[3] # 23년이면 3
year_com = "([0-"+ y_10_before +"]\d|"+y_10+"[0-"+y_1+"])"

# 하면
p_하면4=re.compile("((?<=\D)|^)(?P<y>19\d\d|20"+year_com+")\s?(?P<es>하면)\s?(?P<sn>[0-9]+)") # 4자리 연도
p_하면2=re.compile("((?<=\D)|^)(?P<y>[7-9][0-9]|"+year_com+")\s?(?P<es>하면)\s?(?P<sn>[0-9]+)") # 2자리 연도

# 하단
p_하단4=re.compile("((?<=\D)|^)(?P<y>19\d\d|20"+year_com+")\s?(?P<es>하단)\s?(?P<sn>[0-9]+)") # 4자리 연도
p_하단2=re.compile("((?<=\D)|^)(?P<y>[7-9][0-9]|"+year_com+")\s?(?P<es>하단)\s?(?P<sn>[0-9]+)") # 2자리 연도

# 하
p_하4=re.compile("((?<=\D)|^)(?P<y>19\d\d|20"+year_com+")\s?(?P<es>하)\s?(?P<sn>[0-9]+)") # 4자리 연도
p_하2=re.compile("((?<=\D)|^)(?P<y>[7-9][0-9]|"+year_com+")\s?(?P<es>하)\s?(?P<sn>[0-9]+)") # 2자리 연도

# 복합
p_복합4=re.compile("((?<=\D)|^)(?P<y>19\d\d|20"+year_com+")\s?(?P<es1>하[면단]).?(?P<es2>하?[면단])[^\d]?(?P<sn>[0-9]+)") # 4자리 연도
p_복합2=re.compile("((?<=\D)|^)(?P<y>[7-9][0-9]|"+year_com+")\s?(?P<es1>하[면단]).?(?P<es2>하?[면단])[^\d]?(?P<sn>[0-9]+)") # 2자리 연도

def make_event_formular(case, re_object) :
    if case == 4: 
        event = re_object["y"] + re_object["es"] + re_object["sn"]
        # eSign = re_object["es"]
    elif case == 2:
        if int(re_object["y"])  <= int(y_10 + y_1):
            event = "20" + re_object["y"] + re_object["es"] + re_object["sn"]
        else :
            event = re_object["y"] + re_object["es"] + re_object["sn"] # 1900년대는 사건번호에 19안 붙음
        # eSign = re_object["es"]
    return event

comp = {"하면" : [p_하면4,p_하면2], "하단" : [p_하단4, p_하단2], "하" : [p_하4, p_하2], "복합" : [p_복합4, p_복합2]}

In [37]:
#################################################
검색사건기호 = "하면"
#################################################

추출 = []
for i, row in 미종결메모.iterrows() : 
    
    event_set = set()
    
    # 4연도 사건번호 전부 추출
    res4 = list(comp[검색사건기호][0].finditer(row.메모내용)) 
    if res4 : 
        # 사건번호 규격화 하고, 고유값만 남기기
        
        for r in res4 : 
            form_event = make_event_formular(4, r)
            event_set.add(form_event)
    
    res2 = list(comp[검색사건기호][1].finditer(row.메모내용)) 
    
    # 2연도 사건번호 전부 추출
    if res2 : 
        # 사건번호 규격화 하고, 고유값만 남기기
        for r in res2 : 
            form_event = make_event_formular(4, r)
            event_set.add(form_event)
        
    # 규격화된 사건번호의 고유값 개수에 따라 행을 추가하여 df만들기
    for e in list(event_set) :
        new_row = row.copy()
        new_row["사건번호"] = e
        추출.append(new_row)
    
추출 = pd.concat(추출, axis=1).T

In [43]:
# 파산등록건에 등록되지 않은 사건번호만 남기기
추출_파산등록건 = 추출.merge(파산등록건[["새채무자키","사건번호"]], on=["새채무자키","사건번호"], indicator=True, how='outer')
print(추출_파산등록건._merge.value_counts())
미등록건 = 추출_파산등록건.query('_merge=="left_only"')
미등록건.drop(columns=["_merge"], inplace=True)

_merge
left_only     795
right_only    462
both          363
Name: count, dtype: int64


In [56]:
# 관할법원
p_court = re.compile("(([가-힣]+(법원|지원|지법))|((\s|\b)(서울|대전|대구|부산|광주|수원|\
        |의정부|파주|포천|동두천|가평|연천|철원|인천|김포|강화|용인|오산|광명|\
        |안성|양평|이천|춘천|홍천|양구|삼척|동해|정선|평창|태백|횡성|인제|화천|\
        |고성|양양|금산|세종|보령|서천|예산|아산|태안|당진|(?<![도통문])부여|청양|진천|보은|\
        |단양|음성|옥천|괴산|경산|칠곡|청도|영천|성주|고령|영주|봉화|구미|문경|\
        |예천|청송|군위|울진|영양|양산|울산|창원|마산|김해|함안|의령|사천|남해|\
        |하동|거제|창녕|합천|함양|산청|담양|함평|강진|구례|영광|나주|장성|화순|\
        |곡성|광양|고흥|여수|보성|무안|영암|완도|진도|전주|군산|정읍|남원|진안|\
        |김제|무주|임실|익산|부안|고창|장수|순창|제주|서귀포)[가-힣]*))")

미등록건["관할법원"] = 미등록건.메모내용.str.extract(p_court)[0].str.strip()
미등록건

# 정확한 신청인명은 누락여부 확인하면서 한번더
# 원데이터 > 새채무자키를 기준으로 채무자 이름과 보증인 이름의 고유값만 추가하기
# if 차주만 있음 : 일단 차주로 등록하고 최신화 안 되는 거 나중에 확인
# else 보증인도 있는 경우
#    if 보증인"이름" 있음 : 대상자구분에 보증인 추가, 대상자명에 보증인명 추가
#    else : 차주

# 누락여부 '포함', '누락' search




,메모키,채무자키,메모내용,등록일자,새채무자키,사건번호,관할법원
1,28526243,20411490,*** 외환은행 당사채권 누락 **** 사건번호 : 창원지방법원 2016하면...,2018-06-12,n1005344,2016하면1859,창원지방법원
3,28813543,20413269,[기타] 차주 서울회생법원 2016하면 8125 파산면책 (17.03.27 인용) ...,2019-03-29,n1054434,2016하면8125,서울회생법원
4,28016099,20417607,"[전화통화] 보증인 김희선 파산,면책 서울중앙지방법원 2008하면 22426 ...",2015-09-11,n1005801,2008하면22426,서울중앙지방법원
5,27798487,20417607,[기타] (보증인 김희선) 파산선고 사건번호: 2008하단22426 관련사건번호:...,2013-04-12,n1005801,2008하면22426,서울중앙지방법원
6,27798494,20417607,[기타] (보증인 김희선) 파산/면책 사건번호: 2008하면22426 관련사건번호...,2013-04-12,n1005801,2008하면22426,서울중앙지방법원
...,...,...,...,...,...,...,...
1150,29254271,20413681,"보증인 안혜진 수원지법 2019하단 10976, 2019하면 10976",2020-03-09,n1055753,2019하면10976,수원지법
1151,29179224,20422253,"ㅁ 대전지방법원 2019하단 1660, 19하면 1660 강제집행. 가압류. 가처...",2020-01-03,n1008897,19하면1660,대전지방법원
1155,27801482,20429047,[매입전등록] [기타] 이정숙 수원지법 2012 하단 4821 2012 하면 ...,2013-05-13,n1048985,2012하면4821,수원지법
1156,27761851,20428979,[매입전등록] [기타] 이상림 서울중앙 2011 하단 9502 2011 하면 ...,2011-12-07,n1048761,2011하면9502,서울중앙


관할법원
부산지방법원      63
서울중앙지방법원    55
대구지방법원      44
수원지방법원      44
창원지방법원      43
            ..
서울법원종합청사     1
추심금지울산지법     1
제주           1
장성재          1
중앙지법         1
Name: count, Length: 66, dtype: int64